### Ingestion de la carpeta "movie_company.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo CSV utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
movies_companies_schema = StructType(fields=[
    StructField("movieId", IntegerType(), True),
    StructField("companyId", IntegerType(), True)
    ])


In [0]:
movies_companies_df = spark.read \
    .schema(movies_companies_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/movie_company")

#### Paso 2 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
movies_companies_renamed_df = movies_companies_df \
    .withColumnRenamed("movieId","movie_id") \
    .withColumnRenamed("companyId","company_id")
    

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
movies_companies_final_df = add_ingestion_date(movies_companies_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
display(movies_companies_final_df)

In [0]:
#overwrite_partition(movies_companies_final_df,"movie_silver","movies_companies","file_date")
merge_condition = 'tgt.movie_id = src.movie_id and tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(movies_companies_final_df,'movie_silver','movies_companies',silver_folder_path,merge_condition,'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.movies_companies
GROUP BY file_date;